# HW 2 - Naive Bayes in Hadoop MR
__`MIDS w261: Machine Learning at Scale | UC Berkeley School of Information | Fall 2018`__

In the live sessions for week 2 and week 3 you got some practice designing and debugging Hadoop Streaming jobs. In this homework we'll use Hadoop MapReduce to implement your first parallelized machine learning algorithm: Naive Bayes. As you develop your implementation you'll test it on a small dataset that matches the 'Chinese Example' in the _Manning, Raghavan and Shutze_ reading for Week 2. For the main task in this assignment you'll be working with a small subset of the Enron Spam/Ham Corpus. By the end of this assignment you should be able to:
* __... describe__ the Naive Bayes algorithm including both training and inference.
* __... perform__ EDA on a corpus using Hadoop MR.
* __... implement__ parallelized Naive Bayes.
* __... constrast__ partial, unordered and total order sort and their implementations in Hadoop Streaming.
* __... explain__ how smoothing affects the bias and variance of a Multinomial Naive Bayes model.

As always, your work will be graded both on the correctness of your output and on the clarity and design of your code. __Please refer to the `README` for homework submission instructions.__ 

## Notebook Setup
Before starting, run the following cells to confirm your setup.

In [1]:
# imports
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
!pwd

/media/notebooks


In [3]:
# global vars (paths) - ADJUST AS NEEDED
JAR_FILE = "/usr/lib/hadoop-mapreduce/hadoop-streaming.jar"
HDFS_DIR = "/user/root/HW2"
HOME_DIR = "/media/notebooks" # FILL IN HERE eg. /media/notebooks/Assignments/HW2

In [4]:
# save path for use in Hadoop jobs (-cmdenv PATH={PATH})
from os import environ
PATH  = environ['PATH']

In [5]:
# data path
ENRON = "data/enronemail_1h.txt"

In [22]:
# make the HDFS directory if it doesn't already exist
!hdfs dfs -ls 
!hdfs dfs -mkdir {HDFS_DIR}

Found 1 items
drwxr-xr-x   - root supergroup          0 2021-09-21 16:09 HW2
mkdir: `/user/root/HW2': File exists


In [23]:
!hdfs dfs -ls {HDFS_DIR}

Found 19 items
-rw-r--r--   1 root supergroup        303 2021-09-21 16:02 /user/root/HW2/NBmodel.txt
drwxr-xr-x   - root supergroup          0 2021-09-18 05:39 /user/root/HW2/chinese-output
-rw-r--r--   1 root supergroup        119 2021-09-18 03:05 /user/root/HW2/chineseTest.txt
-rw-r--r--   1 root supergroup        107 2021-09-18 03:05 /user/root/HW2/chineseTrain.txt
drwxr-xr-x   - root supergroup          0 2021-09-18 03:01 /user/root/HW2/eda-output
drwxr-xr-x   - root supergroup          0 2021-09-19 02:46 /user/root/HW2/eda-sort-output
drwxr-xr-x   - root supergroup          0 2021-09-19 00:58 /user/root/HW2/enron-model
-rw-r--r--   1 root supergroup     204559 2021-09-15 05:10 /user/root/HW2/enron.txt
-rw-r--r--   1 root supergroup      41493 2021-09-19 00:56 /user/root/HW2/enron_test.txt
-rw-r--r--   1 root supergroup     163066 2021-09-19 00:56 /user/root/HW2/enron_train.txt
drwxr-xr-x   - root supergroup          0 2021-09-18 20:51 /user/root/HW2/nb-smoothed-chinese-output
drwx

In [18]:
# !hdfs dfs -mkdir {HDFS_DIR}/test

In [48]:
# !hdfs dfs -rm -r {HDFS_DIR}

Deleted /user/root/HW2/smooth-model-part1


# Question 1: Hadoop MapReduce Key Takeaways.  

This assignment will be the only one in which you use Hadoop Streaming to implement a distributed algorithm. The key reason we continue to teach Hadoop streaming is because of the way it forces the programmer to think carefully about what is happening under the hood when you parallelize a calculation. This question will briefly highlight some of the most important concepts that you need to understand about Hadoop Streaming and MapReduce before we move on to Spark next week.   

### Q1 Tasks:

* __a) short response:__ What "programming paradigm" is Hadoop MapReduce based on? What are the main ideas of this programming paradigm and how does MapReduce exemplify these ideas?

* __b) short response:__ What is the Hadoop Shuffle? When does it happen? Why is it potentially costly? Describe one specific thing we can we do to mitigate the cost associated with this stage of our Hadoop Streaming jobs.

* __c) short response:__ In Hadoop Streaming why do the input and output record format of a combiner script have to be the same? [__`HINT`__ _what level of combining does the framework guarantee? what is the relationship between the record format your mapper emits and the format your reducer expects to receive?_]

* __d) short response:__ To what extent can you control the level of parallelization of your Hadoop Streaming jobs? Please be specific.

* __e) short response:__ What change in the kind of computing resources available prompted the creation of parallel computation frameworks like Hadoop? 

### Q1 Student Answers:

> __a)__ Hadoop MapReduce programming model has its roots in functional programming.  It is a declarative programming paradigm, meaning it uses expressions.  In functional code, the output value of a function depends only on the input arguments due to the higher-order function concept (functions that can accept other functions as arguments).  Map and reduce--aka fold, accumulate, compress, or inject--are examples of a higher-order function.  This programming style avoids state change, mutable objects and takes advantage of the embarrassing parallel framework--all critical features.

> __b)__ In many ways, the shuffle is the heart of MapReduce and is where the "magic" happens. The shuffle is the process by which the system performs the sort and transfers the map outputs to the reducers. It is the synchronization process between the Map and Reduce phases. The shuffle is a network hot spot since it occurs after all mappers are complete then transfers the data as key-value pairs to the reducers.  Hadoop identifies which data pair gets allocated to each data node based on the key, at which point the data must be transferred.  Adding a combiner is one way to cut down on network traffic during the shuffle. A combiner will locally aggregate similar keys, therefore reducing the cost associated with the shuffle stage.

> __c)__ A combiner is an aggregation script specified by the programmer.  It combines mapper output records with the same key, turning multiple key-value pairs with the same key into one.  This typically results in a faster shuffle because fewer records need transferring.  It is important to note that Hadoop uses combiners strategically.  Sometimes records are combined before leaving the mapper node or after arriving at a reducer node.  The fact that Hadoop does not guarantee using a combiner on any or all records limits the output of a combiner to having the same format as an input into a reducer.  In other words, the input and output format of a combiner must be the same.  In summary, Hadoop makes smart choices about when to hold data in memory, combine, or spill to disk.  These ultimately improve the shuffle 
efficiency.

> __d)__ The level of parallelization in Hadoop is the culmination of a two-step Map and Reduce process.  The user can indirectly specify the number of map tasks for a given job using the mapred.min.split.size parameter in mapred-site.xml.  This parameter adjusts the input split size, which drives the number of mappers.  Keep in mind the size of the input split should typically match the block size for peak performance.  The user can specify the number of reducers using the -numReduceTasks parameter when invoking a Hadoop job.

> __e)__ The divide and conquer technique of parallel computing became readily available when standard/commodity and affordable architectures such as cloud computing emerged.  Clusters of commodity Linux nodes with CPU, memory, and disk along with gigabit ethernet interconnection prompted the adoption of parallel computation frameworks like Hadoop.  Increasing data size, availability, and the demand to leverage data served as the catalyst for the rapid growth of the data revolution.

# Question 2: MapReduce Design Patterns.  

In the last two live sessions and in your readings from Lin & Dyer you encountered a number of techniques for manipulating the logistics of a MapReduce implementation to ensure that the right information is available at the right time and location. In this question we'll review a few of the key techniques you learned.   

### Q2 Tasks:

* __a) short response:__ What are counters (in the context of Hadoop Streaming)? How are they useful? What kinds of counters does Hadoop provide for you? How do you create your own custom counter?

* __b) short response:__ What are composite keys? How are they useful? How are they related to the idea of custom partitioning?

* __c) short response:__ What is the order inversion pattern? What problem does it help solve? How do we implement it? 

### Q2 Student Answers:

> __a)__ There are two types of Hadoop counters, built-in and custom.  Built-in counters provide a way to measure the progress or number of operations during a MapReduce job.  Counters in Hadoop gather statistics about a job.  They are also helpful in diagnosing implementation issues.  There are five types of built-in counters: Task Counters, FileSystem Counters, FileInputFormat counters, FileOutputFormat Counters, and MapReduce Job Counters.  Specific examples include the following: Task Counters collect specific task information like the number of records read and written during execution.  FileSystemCounters track the number of bytes read and written by the file system.  A Custom Counter is a user-defined counter configurable using Java enum.  It allows users to define custom counting techniques that the mapper or reducer can increment.  

> __b)__ A composite key contains multiple values used for sorting.  It is the combination of a key and a value or several values.  A technique called secondary sorting uses composite keys for sorting multiple fields rather than just one.

> Suppose we have input data containing the highest daily temperature for fifty years in the following format: yyyy ttt.  Our goal is to sort the data into a list of descending years and temperatures.  Secondary sorting using a composite key must be applied to meet this objective.  First, create the composite key of year and temp.  Using custom partitioning based on year only ensures records of the same year get sent to the same reduce task.  Once the grouped years are together, the reducer can use the composite key to sort by year and temp.  To summarize, composite keys allow the ability to sort by more than one value.

> Custom partitioning and composite key techniques are used in tandem to achieve total ordered sort. The composite key allows the programmer to sort based on more than one value.  Custom partitioning assigns keys to partitions with guidance from the programmer, thereby ordering the values across multiple partitions. 

> __c)__ The order inversion pattern allows the user to access the result of a computation in the reducer before processing the data needed for that computation.  This capability makes calculating relative frequencies possible.  Implementation relies on the user creating a new key prefixed by a character guaranteeing it to be the first result the reducer encounters after the sort.


# Question 3: Understanding Total Order Sort

The key challenge in distributed computing is to break a problem into a set of sub-problems that can be performed without communicating with each other. Ideally, we should be able to define an arbirtary number of splits and still get the right result, but that is not always possible. Parallelization becomes particularly challenging when we need to make comparisons between records, for example when sorting. Total Order Sort allows us to order large datasets in a way that enables efficient retrieval of results. Before beginning this assignment, make sure you have read and understand the [Total Order Sort Notebook](https://github.com/UCB-w261/main/tree/master/HelpfulResources/TotalSortGuide/_total-sort-guide-spark2.01-JAN27-2017.ipynb). You can skip the first two MRJob sections, but the rest of section III and all of section IV are **very** important (and apply to Hadoop Streaming) so make sure to read them closely. Feel free to read the Spark sections as well but you won't be responsible for that material until later in the course. To verify your understanding, answer the following questions.

### Q3 Tasks:

* __a) short response:__ What is the difference between a partial sort, an unordered total sort, and a total order sort? From the programmer's perspective, what does total order sort allow us to do that we can't with unordered total? Why is this important with large datasets?

* __b) short response:__ Which phase of a MapReduce job is leveraged to implement Total Order Sort? Which default behaviors must be changed. Why must they be changed?

* __c) short response:__ Describe in words how to configure a Hadoop Streaming job for the custom sorting and partitioning that is required for Total Order Sort.  

* __d) short response:__ Explain why we need to use an inverse hash code function.

* __e) short response:__ Where does this function need to be located so that a Total Order Sort can be performed?

### Q3 Student Answers:

> __a)__ **Partial Sort:** A partial sort outputs several partition files sorted by key-value pairs.  Note that these files are sorted within each file, not ordered across all partitions files.  
> **Unordered Total Sort:** Total unordered sort refers to ordering all key-value pairs based upon a specified key.  This sorting encompasses all partition files, unlike partial sort.  However, there is still some post-processing needed to handle the partition files.  
> **Total Ordered Sort:**  Total order sort not only orders the key-value pairs but assigns them to partitions such that the sequence of records amongst all partitions are ordered. 

> __b)__ Hadoop's architecture only guarantees partial sort in its shuffling and sorting phases.  Additional steps must be incorporated to aid Hadoop's framework during the shuffle and sort phase to achieve total sort.  Three default behaviors must be changed. First, a partition file or function needs to exist inside the mappers. This is where a partition key is generated to instruct Hadoop on how to partition records. Second, internal partition keys inside the reducer need to be dropped. This is where the key used for partitioning is dropped, and the reducer recovers the original (key, value) pairs. Finally, there is a post-processing step to order the partitions. While the partitions are ordered, the partitions keys may not be.

> __c)__ To perform a total order sort, the default shuffle behavior must be changed by providing a custom partitioner, a custom sort specification, and a combiner. 

> __d)__ An inverse hash function is used to preserve partition key ordering.  Its input receives the desired partition index and the total number of partitions, then returns a partition key. This key will hash to the correct partition index when supplied to the Hadoop framework. 

> __e)__ The inverse hash function must be located in the mapper to perform the total ordered sort.

# About the Data
For the main task in this portion of the homework you will train a classifier to determine whether an email represents spam or not. You will train your Naive Bayes model on a 100 record subset of the Enron Spam/Ham corpus available in the HW2 data directory (__`HW2/data/enronemail_1h.txt`__).

__Source:__   
The original data included about 93,000 emails which were made public after the company's collapse. There have been a number raw and preprocessed versions of this corpus (including those available [here](http://www.aueb.gr/users/ion/data/enron-spam/index.html) and [here](http://www.aueb.gr/users/ion/publications.html)). The subset we will use is limited to emails from 6 Enron employees and a number of spam sources. It is part of [this data set](http://www.aueb.gr/users/ion/data/enron-spam/) which was created by researchers working on personlized Bayesian spam filters. Their original publication is [available here](http://www.aueb.gr/users/ion/docs/ceas2006_paper.pdf). __`IMPORTANT!`__ _For this homework please limit your analysis to the 100 email subset which we provide. No need to download or run your analysis on any of the original datasets, those links are merely provided as context._

__Preprocessing:__  
For their work, Metsis et al. (the authors) appeared to have pre-processed the data, not only collapsing all text to lower-case, but additionally separating "words" by spaces, where "words" unfortunately include punctuation. As a concrete example, the sentence:  
>  `Hey Jon, I hope you don't get lost out there this weekend!`  

... would have been reduced by Metsis et al. to the form:  
> `hey jon , i hope you don ' t get lost out there this weekend !` 

... so we have reverted the data back toward its original state, removing spaces so that our sample sentence would now look like:
> `hey jon, i hope you don't get lost out there this weekend!`  

Thus we have at least preserved contractions and other higher-order lexical forms. However, one must be aware that this reversion is not complete, and that some object (specifically web sites) will be ill-formatted, and that all text is still lower-cased.


__Format:__   
All messages are collated to a tab-delimited format:  

>    `ID \t SPAM \t SUBJECT \t CONTENT \n`  

where:  
>    `ID = string; unique message identifier`  
    `SPAM = binary; with 1 indicating a spam message`  
    `SUBJECT = string; title of the message`  
    `CONTENT = string; content of the message`   
    
Note that either of `SUBJECT` or `CONTENT` may be "NA", and that all tab (\t) and newline (\n) characters have been removed from both of the `SUBJECT` and `CONTENT` columns.  

In [26]:
!pwd

/media/notebooks


In [27]:
# take a look at the first 100 characters of the first 5 records (RUN THIS CELL AS IS)
!head -n 5 /media/notebooks/{ENRON} | cut -c-100

0001.1999-12-10.farmer	0	 christmas tree farm pictures	NA
0001.1999-12-10.kaminski	0	 re: rankings	 thank you.
0001.2000-01-17.beck	0	 leadership development pilot	" sally:  what timing, ask and you shall receiv
0001.2000-06-06.lokay	0	" key dates and impact of upcoming sap implementation over the next few week
0001.2001-02-07.kitchen	0	 key hr issues going forward	 a) year end reviews-report needs generating 


In [32]:
# see how many messages/lines are in the file 
# this number may be off by 1 if the last line doesn't end with a newline
!wc -l {HOME_DIR}/{ENRON}

100 /media/notebooks/data/enronemail_1h.txt


In [30]:
# make the HDFS directory if it doesn't already exist
!hdfs dfs -mkdir {HDFS_DIR}

mkdir: `/user/root/HW2': File exists


In [33]:
# load the data into HDFS (RUN THIS CELL AS IS)
!hdfs dfs -copyFromLocal {HOME_DIR}/{ENRON} {HDFS_DIR}/enron.txt

In [41]:
# view file structure
!hdfs dfs -ls {HDFS_DIR}

Found 1 items
-rw-r--r--   1 root supergroup     204559 2021-09-15 05:10 /user/root/HW2/enron.txt


In [15]:
# output two lines from enron.txt stored in hdfs
!hdfs dfs -cat {HDFS_DIR}/enron.txt | head -2

0001.1999-12-10.farmer	0	 christmas tree farm pictures	NA
0001.1999-12-10.kaminski	0	 re: rankings	 thank you.
cat: Unable to write to output stream.


# Question 4:  Enron Ham/Spam EDA.
Before building our classifier, lets get aquainted with our data. In particular, we're interested in which words occur more in spam emails than in legitimate ("ham") emails. In this question you'll implement two Hadoop MapReduce jobs to count and sort word occurrences by document class. You'll also learn about two new Hadoop streaming parameters that will allow you to control how the records output from your mappers are partitioned for reducing on separate nodes. 

__`IMPORTANT NOTE:`__ For this question and all subsequent items, you should include both the subject and the body of the email in your analysis (i.e. concatetate them to get the 'text' of the document).

### Q4 Tasks:
* __a) code:__ Complete the missing components of the code in __`EnronEDA/mapper.py`__ and __`EnronEDA/reducer.py`__ to create a Hadoop  MapReduce job that counts how many times each word in the corpus occurs in an email for each class. Pay close attention to the data format specified in the docstrings of these scripts _-- there are a number of ways to accomplish this task, we've chosen this format to help illustrate a technique in `part e`_. Run the provided unit tests to confirm that your code works as expected, then run the provided Hadoop Streaming command to apply your analysis to the Enron data.


* __b) code + short response:__ How many times does the word "__assistance__" occur in each class? (`HINT:` Use a `grep` command to read from the results file you generated in '`a`' and then report the answer in the space provided.)


* __c) short response:__ Would it have been possible to add some sorting parameters to the Hadoop streaming command that would cause our `part a` results to be sorted by count? Explain why or why not. (`HINT:` This question demands an understanding of the sequence of the phases of MapReduce.)


* __d) code + short response:__ Write a second Hadoop MapReduce job to sort the output of `part a` first by class and then by count. Run your job and save the results to a local file. Then describe in words how you would go about printing the top 10 words in each class given this sorted output. (`HINT 1:` _remember that you can simply pass the `part a` output directory to the input field of this job; `HINT 2:` since this task is just reodering the records from `part a` we don't need to write a mapper or reducer, just use `/bin/cat` for both_)


* __e) code:__ A more efficient alternative to '`grep`-ing' for the top 10 words in each class would be to use the Hadoop framework to separate records from each class into its own partition so that we can just read the top lines in each. Rewrite your job from ` part d` to specify 2 reduce tasks and to tell Hadoop to partition based on the second field (which indicates spam/ham in our data). Your code should maintain the secondary sort -- that is each partition should list words from most to least frequent.

### Q4 Student Answers:
> __b)__ The word assistance appears twice in the ham class and eight times in the spam class.  See output below:

|word| class | count |
|---|:---:|:---:|
| assistance | 0 | 2
| assistance | 1 | 8

> __c)__ Yes, a composite key and custom partitioning (if using more than one reducer) must be added to the Hadoop command to produce an output sorted by count.

> __d)__ A head -10 command prints the top ten words of class 0.  To print the top ten words of class 1, use the head command piped into a grep command with a parameter for class 1.  This sequence will display the top ten words of each class from the results_sorted.txt file.  

In [40]:
# part a - do your work in the provided scripts then RUN THIS CELL AS IS
!chmod a+x EnronEDA/mapper.py
!chmod a+x EnronEDA/reducer.py

In [4]:
# part a - unit test EnronEDA/mapper.py (RUN THIS CELL AS IS)
!echo -e "d1	1	title	body\nd2	0	title	body" | EnronEDA/mapper.py

title	1	1
body	1	1
title	0	1
body	0	1


In [5]:
# part a - unit test EnronEDA/reducer.py (RUN THIS CELL AS IS)
!echo -e "one	1	1\none	0	1\none	0	1\ntwo	0	1" | EnronEDA/reducer.py

one	0	2
one	1	1
two	0	1


In [48]:
# part a - clear output directory in HDFS (RUN THIS CELL AS IS)
!hdfs dfs -rm -r {HDFS_DIR}/eda-output

Deleted /user/root/HW2/eda-output


In [49]:
# part a - Hadoop streaming job (RUN THIS CELL AS IS)
!hadoop jar {JAR_FILE} \
  -files EnronEDA/reducer.py,EnronEDA/mapper.py \
  -mapper mapper.py \
  -reducer reducer.py \
  -input {HDFS_DIR}/enron.txt \
  -output {HDFS_DIR}/eda-output \
  -numReduceTasks 2 \
  -cmdenv PATH={PATH}

packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.16.2.jar] /tmp/streamjob4188820668399500647.jar tmpDir=null
21/09/22 04:02:32 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/09/22 04:02:32 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/09/22 04:02:32 INFO mapred.FileInputFormat: Total input paths to process : 1
21/09/22 04:02:32 INFO mapreduce.JobSubmitter: number of splits:2
21/09/22 04:02:33 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1632276588378_0008
21/09/22 04:02:33 INFO impl.YarnClientImpl: Submitted application application_1632276588378_0008
21/09/22 04:02:33 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1632276588378_0008/
21/09/22 04:02:33 INFO mapreduce.Job: Running job: job_1632276588378_0008
21/09/22 04:02:40 INFO mapreduce.Job: Job job_1632276588378_0008 running in uber mode : false
21/09/22 04:02:40 INFO mapreduce.Job:  map 0% reduce 0%
21/09

In [15]:
# part a - retrieve results from HDFS & copy them into a local file (RUN THIS CELL AS IS)
!hdfs dfs -cat {HDFS_DIR}/eda-output/part-0000* > EnronEDA/results.txt

In [123]:
# part b - write your grep command here
!grep "assistance" EnronEDA/results.txt

assistance	0	2
assistance	1	8


In [128]:
!head -10 EnronEDA/results_sort.txt

to	0	267	
and	0	213	
the	0	201	
a	0	131	
in	0	127	
enron	0	98	
on	0	97	
of	0	79	
be	0	76	
com	0	74	


In [54]:
# part d - clear the output directory in HDFS (RUN THIS CELL AS IS)
!hdfs dfs -rm -r {HDFS_DIR}/eda-sort-output

Deleted /user/root/HW2/eda-sort-output


In [55]:
# part d - write your Hadoop streaming job here
!hadoop jar {JAR_FILE} \
  -D stream.num.map.output.key.fields=3 \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D mapreduce.partition.keycomparator.options="-k2n,2nr -k3nr" \
  -mapper /bin/cat \
  -reducer /bin/cat \
  -input {HDFS_DIR}/eda-output \
  -output {HDFS_DIR}/eda-sort-output \
  -numReduceTasks 1 \
  -cmdenv PATH={PATH}

packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.16.2.jar] /tmp/streamjob4755914085112699868.jar tmpDir=null
21/09/22 04:07:28 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/09/22 04:07:28 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/09/22 04:07:29 INFO mapred.FileInputFormat: Total input paths to process : 2
21/09/22 04:07:29 INFO mapreduce.JobSubmitter: number of splits:2
21/09/22 04:07:30 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1632276588378_0010
21/09/22 04:07:30 INFO impl.YarnClientImpl: Submitted application application_1632276588378_0010
21/09/22 04:07:30 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1632276588378_0010/
21/09/22 04:07:30 INFO mapreduce.Job: Running job: job_1632276588378_0010
21/09/22 04:07:36 INFO mapreduce.Job: Job job_1632276588378_0010 running in uber mode : false
21/09/22 04:07:36 INFO mapreduce.Job:  map 0% reduce 0%
21/09

In [56]:
# output results into a text file
!hdfs dfs -cat {HDFS_DIR}/eda-sort-output/part-0000* > EnronEDA/results_sorted.txt

In [66]:
print("Class 0 Top Ten")
!awk '$2==0' < EnronEDA/results_sorted.txt | head -10
print("Class 1 Top Ten")
!awk '$2==1' < EnronEDA/results_sorted.txt | head -10

Class 0 Top Ten
the	0	549	
to	0	398	
ect	0	382	
and	0	278	
of	0	230	
hou	0	206	
a	0	196	
in	0	182	
for	0	170	
on	0	135	
Class 1 Top Ten
the	1	698	
to	1	566	
and	1	392	
your	1	357	
a	1	347	
you	1	345	
of	1	336	
in	1	236	
for	1	204	
com	1	153	


In [44]:
# part e - clear the output directory in HDFS (RUN THIS CELL AS IS)
!hdfs dfs -rm -r {HDFS_DIR}/eda-sort-output

Deleted /user/root/HW2/eda-sort-output


In [50]:
# part e - clear the output directory in HDFS (RUN THIS CELL AS IS)
!hdfs dfs -rm -r {HDFS_DIR}/eda-sort-output
# part e - write your Hadoop streaming job here
!hadoop jar {JAR_FILE} \
  -D stream.num.map.output.key.fields=3 \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D mapreduce.partition.keycomparator.options="-k2n,2nr -k3nr" \
  -D mapreduce.partition.keypartitioner.options="-k2,2n" \
  -mapper /bin/cat \
  -reducer /bin/cat \
  -input {HDFS_DIR}/eda-output \
  -output {HDFS_DIR}/eda-sort-output \
  -numReduceTasks 2 \
  -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner \
  -cmdenv PATH={PATH}

Deleted /user/root/HW2/eda-sort-output
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.16.2.jar] /tmp/streamjob1232426668069957632.jar tmpDir=null
21/09/22 04:03:06 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/09/22 04:03:06 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/09/22 04:03:06 INFO mapred.FileInputFormat: Total input paths to process : 2
21/09/22 04:03:06 INFO mapreduce.JobSubmitter: number of splits:2
21/09/22 04:03:07 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1632276588378_0009
21/09/22 04:03:07 INFO impl.YarnClientImpl: Submitted application application_1632276588378_0009
21/09/22 04:03:07 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1632276588378_0009/
21/09/22 04:03:07 INFO mapreduce.Job: Running job: job_1632276588378_0009
21/09/22 04:03:13 INFO mapreduce.Job: Job job_1632276588378_0009 running in uber mode : false
21/09/22 04:03:13 INFO

In [51]:
# part e - view the top 10 records from each partition (RUN THIS CELL AS IS)
for idx in range(2):
    print(f"\n===== part-0000{idx}=====\n")
    !hdfs dfs -cat {HDFS_DIR}/eda-sort-output/part-0000{idx} | head


===== part-00000=====

the	0	549	
to	0	398	
ect	0	382	
and	0	278	
of	0	230	
hou	0	206	
a	0	196	
in	0	182	
for	0	170	
on	0	135	
cat: Unable to write to output stream.

===== part-00001=====

the	1	698	
to	1	566	
and	1	392	
your	1	357	
a	1	347	
you	1	345	
of	1	336	
in	1	236	
for	1	204	
com	1	153	
cat: Unable to write to output stream.


__Expected output:__
<table>
<th>part-00000:</th>
<th>part-00001:</th>
<tr><td><pre>
the	0	549	
to	0	398	
ect	0	382	
and	0	278	
of	0	230	
hou	0	206	
a	0	196	
in	0	182	
for	0	170	
on	0	135
</pre></td>
<td><pre>
the	1	698	
to	1	566	
and	1	392	
your	1	357	
a	1	347	
you	1	345	
of	1	336	
in	1	236	
for	1	204	
com	1	153
</pre></td></tr>
</table>

# Question 5: Counters and Combiners.
Tuning the number of mappers & reducers is helpful to optimize very large distributed computations. Doing so successfully requires a thorough understanding of the data size at each stage of the job. As you learned in the week3 live session, counters are an invaluable resource for understanding this kind of detail. In this question, we will take the EDA performed in Question 4 as an opportunity to illustrate some related concepts.

### Q5 Tasks:
* __a) short response:__ Read the Hadoop output from your job in Question 4a to report how many records are emitted by the mappers and how many records are received be the reducers. In the context of word counting what does this number represent practically?

* __b) code:__ Note that we wrote the reducer in question 4a such that the input and output record format is identical. This makes it easy to use the same reducer script as a combiner. In the space provided below, write the Hadoop Streaming command to re-run your job from question 4a with this combining added.

* __c) short response__: Report the number of records emitted by your mappers in part b and the number of records received by your reducers. Compare your results here to what you saw in part a. Explain.

* __d) short response__: Describe a scenario where using a combiner would _NOT_ improve the efficiency of the shuffle stage. Explain. [__`BONUS:`__ how does increasing the number of mappers affect the usefulness of a combiner?]

### Q5 Student Answers:
> __a)__ The mapper emits 31,490 records.  The reducer receives 31,490 records.  This number represents the number of words in the corpus.     
```
4a MapReduce Output
    .....
    Map output records=31490  
    .....  
    Reduce input records=31490   
    .....  
    Total committed heap usage (bytes)=1206910976  
```

> __c)__ The mapper emits 31,490 records.  The reducer receives 7,648 records.  The combiner summed records with similar keys, transferring fewer data (records) from the mapper to the reducer.  As we learned, this results in communication cost reduction during the shuffle.  

```
5b MapReduce Output
    .....
    Map output records=31490  
    .....  
    Reduce input records=7648 
    .....  
    Total committed heap usage (bytes)=1206910976  
```

> __d)__ If a corpus contained every word in the English dictionary only once, a combiner would not improve the efficiency during the shuffle stage.  Increasing the number of mappers decreases the benefits of combiners; it results in less aggregation, which results in more records shuffled from mappers to reducers.

In [24]:
!chmod a+x EnronEDA/combiner.py

In [25]:
# part b - clear output directory in HDFS (RUN THIS CELL AS IS)
!hdfs dfs -rm -r {HDFS_DIR}/eda-output
# part b - write your Hadoop streaming job here
!hadoop jar {JAR_FILE} \
  -files EnronEDA/reducer.py,EnronEDA/mapper.py,EnronEDA/combiner.py \
  -mapper mapper.py \
  -combiner combiner.py \
  -reducer reducer.py \
  -input {HDFS_DIR}/enron.txt \
  -output {HDFS_DIR}/eda-output \
  -numReduceTasks 1 \
  -cmdenv PATH={PATH}

packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.16.2.jar] /tmp/streamjob5210001868591146956.jar tmpDir=null
21/09/18 03:01:30 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/09/18 03:01:30 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/09/18 03:01:31 INFO mapred.FileInputFormat: Total input paths to process : 1
21/09/18 03:01:31 INFO mapreduce.JobSubmitter: number of splits:2
21/09/18 03:01:31 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1631927043264_0003
21/09/18 03:01:31 INFO impl.YarnClientImpl: Submitted application application_1631927043264_0003
21/09/18 03:01:31 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1631927043264_0003/
21/09/18 03:01:31 INFO mapreduce.Job: Running job: job_1631927043264_0003
21/09/18 03:01:37 INFO mapreduce.Job: Job job_1631927043264_0003 running in uber mode : false
21/09/18 03:01:37 INFO mapreduce.Job:  map 0% reduce 0%
21/09

# Question 6: Document Classification Task Overview.
The week 2 assigned reading from Chapter 13 of _Introduction to Information Retrieval_ by Manning, Raghavan and Schutze provides a thorough introduction to the document classification task and the math behind Naive Bayes. In this question we'll use the example from Table 13.1 (reproduced below) to 'train' an unsmoothed Multinomial Naive Bayes model and classify a test document by hand.

<table>
<th>DocID</th>
<th>Class</th>
<th>Subject</th>
<th>Body</th>
<tr><td>Doc1</td><td>1</td><td></td><td>Chinese Beijing Chinese</td></tr>
<tr><td>Doc2</td><td>1</td><td></td><td>Chinese Chinese Shanghai</td></tr>
<tr><td>Doc3</td><td>1</td><td></td><td>Chinese Macao</td></tr>
<tr><td>Doc4</td><td>0</td><td></td><td>Tokyo Japan Chinese</td></tr>
</table>

### Q6 Tasks:
* __a) short response:__ Equation 13.3 in Manning, Raghavan and Shutze shows how a Multinomial Naive Bayes model classifies a document. It predicts the class, $c$, for which the estimated conditional probability of the class given the document's contents,  $\hat{P}(c|d)$, is greatest. In this equation what two pieces of information are required to calculate  $\hat{P}(c|d)$? Your answer should include both mathematical notatation and verbal explanation.


* __b) short response:__ The Enron data includes two classes of documents: `spam` and `ham` (they're actually labeled `1` and `0`). In plain English, explain what  $\hat{P}(c)$ and   $\hat{P}(t_{k} | c)$ mean in the context of this data. How will we would estimate these values from a training corpus? How many passes over the data would we need to make to retrieve this information for all classes and all words?


* __c) hand calculations:__ Above we've reproduced the document classification example from the textbook (we added an empty subject field to mimic the Enron data format). Remember that the classes in this "Chinese Example" are `1` (about China) and `0` (not about China). Calculate the class priors and the conditional probabilities for an __unsmoothed__ Multinomial Naive Bayes model trained on this data. Show the calculations that lead to your result using markdown and $\LaTeX$ in the space provided or by embedding an image of your hand written work. [`NOTE:` _Your results should NOT match those in the text -- they are training a model with +1 smoothing you are training a model without smoothing_]


* __d) hand calculations:__ Use the model you trained to classify the following test document: `Chinese Chinese Chinese Tokyo Japan`. Show the calculations that lead to your result using markdown and   $\LaTeX$ in the space provided or by embedding an image of your hand written work.


* __e) short response:__ Compare the classification you get from this unsmoothed model in `d`/`e` to the results in the textbook's "Example 1" which reflects a model with Laplace plus 1 smoothing. How does smoothing affect our inference?

### Q6 Student Answers:
> __a)__ The two pieces of information needed to calculate $\hat{P}(c|d)$ 
are the $\hat{P}(t_k|c)$ and $\hat{P}(c)$.  $\hat{P}(t_k|c)$ is the conditional probability of term $t_k$ occurring in a document of class c.  $\hat{P}(c)$ is the prior probability of a document occurring in class c.

> __b)__ The prior probability $\hat{P}(c)$, as it relates to the Enron data, is the probability that a document will be spam or ham.  This probability is called the prior probability because it is derived from present data, although it is continuously updated as more data becomes available.  The conditional probability $\hat{P}(t_k|c)$ is the probability of a term k occurring in a document given the document is either spam or ham. 

> To calculate both terms I will use the maximum likelihood estimate, which is the relative frequency corresponding to the most likely value of each parameter given the training data.  

>For prior probability $\hat{P}(c)$, I will use the fromula  
$\hat{P}(c)$ = $\frac{N_c}{N}$  
    
> where Nc is the number of documents in class spam or ham, and N is the total number of documents.

> For conditional probability $\hat{P}(tk|c)$, I will use the formula  
$\hat{P}(t_k|c)$ = $\frac{T_{ct}}{\sum_{t'\in{V}}T_{ct'}}$   
    
> where $T_{ct}$ is the number of occurrences of term $t$ in training documents from class $c$, including multiple occurrences of a term in a document. Recall that Naive Bayes assumes independence.  

> This information can be calculated in one pass using one reducer and order inversion, which allows the programmer to calculate the total number of documents and terms then make the necessary calculations.  However, this framework is not scalable. I prefer two passes over the data—one to calculate vocabulary size and the other to compute conditional probabilities using multiple reducers.  

> __c)__ Calculations



> Prior Probabilities  

> $\hat{P}(c)$ = $\frac{3}{4}$ &nbsp; &nbsp; &nbsp; $\hat{P}(c)$ = $\frac{1}{4}$  


> Likelihoods  
    
> $\hat{P}(chinese|c)$ = $\frac{5}{8}$ &nbsp; &nbsp; &nbsp; $\hat{P}(chinese|\bar{c})$ = $\frac{1}{3}$ &nbsp; &nbsp; &nbsp; $\hat{P}(beijing|c)$ = $\frac{1}{8}$ &nbsp; &nbsp; &nbsp; $\hat{P}(beijing|\bar{c})$ = $0$ &nbsp; &nbsp; &nbsp; $\hat{P}(shanghai|c)$ = $\frac{1}{8}$ &nbsp; &nbsp; &nbsp; $\hat{P}(shanghai|\bar{c})$ = $0$  
    
> $\hat{P}(macao|c)$ = $\frac{1}{8}$ &nbsp; &nbsp; &nbsp; $\hat{P}(macao|\bar{c})$ = $0$ &nbsp; &nbsp; &nbsp; $\hat{P}(japan|c)$ = $0$ &nbsp; &nbsp; &nbsp; $\hat{P}(japan|\bar{c})$ = $\frac{1}{3}$ &nbsp; &nbsp; &nbsp; $\hat{P}(tokyo|c)$ = $0$ &nbsp; &nbsp; &nbsp; $\hat{P}(tokyo|\bar{c})$ = $\frac{1}{3}$

 > __d)__ This document is categorized as not chinese when using an unsmoothed Naive Bayes classifier.  The result reveals the unfavorable outcome of an unsmoothed Naive Bayes model introduced to words not present in the training corpus.  
Calculations:

> $\hat{P}(c|doc)$ = $\hat{P}(c) \prod \hat{P}(t_k|c)$ = $\frac{3}{4} \times (\frac{5}{8} \times \frac{5}{8} \times \frac{5}{8} \times 0 \times 0) = 0$   

> $\hat{P}(\bar{c}|doc)$ = $\hat{P}(\bar{c}) \prod \hat{P}(t_k|\bar{c})$ = $\frac{1}{4} \times (\frac{1}{3} \times \frac{1}{3} \times \frac{1}{3} \times \frac{1}{3} \times \frac{1}{3}) = \frac{1}{972} $

> __e)__ The Laplace smoothing changes the classification of the document from not China to China.  The result exemplifies the benefits of using a smoothed Naive Bayes model vs. an unsmoothed one.  Again, the unsmoothed model does not perform well when presented with words not found in the training data.

In [68]:
# part d/e - if you didn't write out your calcuations above, embed a picture of them here:
from IPython.display import Image
# Image(filename="path-to-hand-calulations-image.png")

# Question 7: Naive Bayes Inference.
In the next two questions you'll write code to parallelize the Naive Bayes calculations that you performed above. We'll do this in two phases: one MapReduce job to perform training and a second MapReduce to perform inference. While in practice we'd need to train a model before we can use it to classify documents, for learning purposes we're going to develop our code in the opposite order. By first focusing on the pieces of information/format we need to perform the classification (inference) task you should find it easier to develop a solid implementation for training phase when you get to question 8 below. In both of these questions we'll continue to use the Chinese example corpus from the textbook to help us test our MapReduce code as we develop it. Below we've reproduced the corpus, test set and model in text format that matches the Enron data.

### Q7 Tasks:
* __a) short response:__ run the provided cells to create the example files and load them in to HDFS. Then take a closer look at __`NBmodel.txt`__. This text file represents a Naive Bayes model trained (with Laplace +1 smoothing) on the example corpus. What are the 'keys' and 'values' in this file? Which record means something slightly different than the rest? The value field of each record includes two numbers which will be helpful for debugging but which we don't actually need to perform inference -- what are they? [`HINT`: _This file represents the model from Example 13.1 in the textbook, if you're having trouble getting oriented try comparing our file to the numbers in that example._]


* __b) short response:__ When performing Naive Bayes in practice instead of multiplying the probabilities (as in equation 13.3) we add their logs (as in equation 13.4). Why do we choose to work with log probabilities? If we had an unsmoothed model, what potential error could arise from this transformation?


* __c) short response:__ Documents 6 and 8 in the test set include a word that did not appear in the training corpus (and as a result does not appear in the model). What should we do at inference time when we need a class conditional probability for this word?


* __d) short response:__ The goal of our MapReduce job is to stream over the test set and classify each document by peforming the calculation from equation 13.4. To do this we'll load the model file (which contains the probabilities for equation 13.4) into memory on the nodes where we do our mapping. This is called an in-memory join. Does loading a model 'state' like this depart from the functional programming principles? Explain why or why not. From a scability perspective when would this kind of memory use be justified? when would it be unwise?


* __e) code:__ Complete the code in __`NaiveBayes/classify_mapper.py`__. Read the docstring carefully to understand how this script should work and the format it should return. Run the provided unit tests to confirm that your script works as expected then write a Hadoop streaming job to classify the Chinese example test set. [`HINT 1:` _you shouldn't need a reducer for this one._ `HINT 2:` _Don't forget to add the model file to the_ `-files` _parameter in your Hadoop streaming job so that it gets shipped to the mapper nodes where it will be accessed by your script._]


* __f) short response:__ In our test example and in the Enron data set we have fairly short documents. Since these fit fine in memory on a mapper node we didn't need a reducer and could just do all of our calculations in the mapper. However with much longer documents (eg. books) we might want a higher level of parallelization -- for example we might want to process parts of a document on different nodes. In this hypothetical scenario how would our algorithm design change? What could the mappers still do? What key-value structure would they emit? What would the reducers have to do as a last step?

### Q7 Student Answers:
> __a)__ This file contains two categories of keys: words and ClassPriors.  The word category keys have four values.  The first two values correspond to the number of times the word is present in the corpus with document classification $\bar{c}$ and $c$.  The following two values are the Laplace smoothed conditional probabilities of the particular word for each class.    

> The 'ClassPriors' key has different corresponding values than the rest of the records.  The first two values are the number of times a document was classified $\bar{c}$ and $c$.  The following two values are the class priors, the probability a document is $\bar{c}$ and $c$.  

> The values not needed for inference are the counts.  Recall that the prior probability and conditional probabilities are the only required values to classify a document.

> __b)__ Adding logarithms, opposed to multiplying probabilities, is an arithmetic method used to prevent a floating point underflow error when computing probabilities.  A floating point underflow error occurs when a calculation result is of a smaller absolute value than a computer can represent in memory.  

> One issue with this approach when using an unsmoothed model is log(0) yields an error. 

> __c)__ If a document did not occur in the corpus at training time, I would set the conditional probability to one.  Doing so would effectively disregard the word, but it would not render the other training calculation useless during classification. 

> __d)__ Loading the model state does not depart from functional programming.  If anything, it leverages functional programming by allowing the user to load different models to perform calculations on other corpora.  In regards to scalability, this action is justified given sufficient memory is available--it speeds up computation.  It's unwise to use this approach on a large corpus; resource availability is always a constraint and should be considered.  

> __e)__ Complete the coding portion of this question before answering 'f'.

> __f)__ The algorithm design approach for larger documents should utilize custom partitioning to distribute keys evenly across reducers.  Moreover, rather than loading a model, the mappers should calculate the total words for each class in a document and emit them as word - class0WordCount, class1WordCount key-value pairs along with class0 and class1 document counts.  These records should then feed into a reducer which calculates the priors and conditional probabilities. 


Run these cells to create the example corpus and model.

In [33]:
%%writefile NaiveBayes/chineseTrain.txt
D1	1		Chinese Beijing Chinese
D2	1		Chinese Chinese Shanghai
D3	1		Chinese Macao
D4	0		Tokyo Japan Chinese

Writing NaiveBayes/chineseTrain.txt


In [34]:
%%writefile NaiveBayes/chineseTest.txt
D5	1		Chinese Chinese Chinese Tokyo Japan
D6	1		Beijing Shanghai Trade
D7	0		Japan Macao Tokyo
D8	0		Tokyo Japan Trade

Writing NaiveBayes/chineseTest.txt


In [35]:
%%writefile NBmodel.txt
beijing	0.0,1.0,0.111111111111,0.142857142857
chinese	1.0,5.0,0.222222222222,0.428571428571
tokyo	1.0,0.0,0.222222222222,0.0714285714286
shanghai	0.0,1.0,0.111111111111,0.142857142857
ClassPriors	1.0,3.0,0.25,0.75
japan	1.0,0.0,0.222222222222,0.0714285714286
macao	0.0,1.0,0.111111111111,0.142857142857

Writing NBmodel.txt


In [40]:
# load the data files into HDFS
!hdfs dfs -copyFromLocal NaiveBayes/chineseTrain.txt {HDFS_DIR}
!hdfs dfs -copyFromLocal NaiveBayes/chineseTest.txt {HDFS_DIR}

copyFromLocal: `/user/root/HW2/chineseTrain.txt': File exists
copyFromLocal: `/user/root/HW2/chineseTest.txt': File exists


Your work for `part e` starts here:

In [41]:
# part e - do your work in NaiveBayes/classify_mapper.py first, then run this cell.
!chmod a+x NaiveBayes/classify_mapper.py

In [72]:
!cat NaiveBayes/classify_mapper.py

#!/usr/bin/env python
"""
Mapper for Naive Bayes Inference.
INPUT:
    ID \t true_class \t subject \t body \n
OUTPUT:
    ID \t true_class \t logP(ham|doc) \t logP(spam|doc) \t predicted_class
SUPPLEMENTAL FILE: 
    This script requires a trained Naive Bayes model stored 
    as NBmodel.txt in the current directory. The model should 
    be a tab separated file whose records look like:
        WORD \t ham_count,spam_count,P(word|ham),P(word|spam)
        
Instructions:
    We have loaded the supplemental file and taken the log of 
    each conditional probability in the model. We also provide
    the code to tokenize the input lines for you. Keep in mind 
    that each 'line' of this file represents a unique document 
    that we wish to classify. Fill in the missing code to get
    the probability of each class given the words in the document.
    Remember that you will need to handle the case where you
    encounter a word that is not represented in the model.
"""
import os
import r

In [45]:
# part e - unit test NaiveBayes/classify_mapper.py (RUN THIS CELL AS IS)
!cat NaiveBayes/chineseTest.txt | NaiveBayes/classify_mapper.py | column -t

d5  1  -8.90668134500626   -8.10769031284611   1
d6  1  -5.780743515794329  -4.179502370564408  1
d7  0  -6.591673732011658  -7.511706880737811  0
d8  0  -4.394449154674438  -5.565796731681498  0


In [46]:
# part e - clear the output directory in HDFS (RUN THIS CELL AS IS)
!hdfs dfs -rm -r {HDFS_DIR}/chinese-output
# part e - write your Hadooop streaming job here
!hadoop jar {JAR_FILE} \
  -files NaiveBayes/classify_mapper.py,NBmodel.txt \
  -mapper classify_mapper.py \
  -input {HDFS_DIR}/chineseTest.txt \
  -output {HDFS_DIR}/chinese-output \
  -cmdenv PATH={PATH}

packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.16.2.jar] /tmp/streamjob2776376637524329372.jar tmpDir=null
21/09/18 05:38:59 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/09/18 05:38:59 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/09/18 05:39:00 INFO mapred.FileInputFormat: Total input paths to process : 1
21/09/18 05:39:00 INFO mapreduce.JobSubmitter: number of splits:2
21/09/18 05:39:00 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1631927043264_0004
21/09/18 05:39:00 INFO impl.YarnClientImpl: Submitted application application_1631927043264_0004
21/09/18 05:39:00 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1631927043264_0004/
21/09/18 05:39:00 INFO mapreduce.Job: Running job: job_1631927043264_0004
21/09/18 05:39:06 INFO mapreduce.Job: Job job_1631927043264_0004 running in uber mode : false
21/09/18 05:39:06 INFO mapreduce.Job:  map 0% reduce 0%
21/09

In [47]:
# part e - retrieve test set results from HDFS (RUN THIS CELL AS IS)
!hdfs dfs -cat {HDFS_DIR}/chinese-output/part-000* > NaiveBayes/chineseResults.txt

In [48]:
# part e - take a look (RUN THIS CELL AS IS)
!cat NaiveBayes/chineseResults.txt | column -t

d5  1  -8.90668134500626   -8.10769031284611   1
d6  1  -5.780743515794329  -4.179502370564408  1
d7  0  -6.591673732011658  -7.511706880737811  0
d8  0  -4.394449154674438  -5.565796731681498  0


<table>
<th> Expected output for the test set:</th>
<tr align=Left><td><pre>
d5	1	-8.90668134	-8.10769031	1
d6	1	-5.78074351	-4.17950237	1
d7	0	-6.59167373	-7.51170688	0
d8	0	-4.39444915	-5.56579673	0
</pre></td><tr>
</table>

# Question 8: Naive Bayes Training.
In Question 7 we used a model that we had trained by hand. Next we'll develop the code to do that same training in parallel, making it suitable for use with larger corpora (like the Enron emails). The end result of the MapReduce job you write in this question should be a model text file that looks just like the example (`NBmodel.txt`) that we created by hand above.

To refresh your memory about the training process take a look at  `6a` and `6b` where you described the pieces of information you'll need to collect in order to encode a Multinomial Naive Bayes model. We now want to retrieve those pieces of information while streaming over a corpus. The bulk of the task will be very similar to the word counting excercises you've already done but you may want to consider a slightly different key-value record structure to efficiently tally counts for each class. 

The most challenging (interesting?) design question will be how to retrieve the totals (# of documents and # of words in documents for each class). Of course, counting these numbers is easy. The hard part is the timing: you'll need to make sure you have the counts totalled up _before_ you start estimating the class conditional probabilities for each word. It would be best (i.e. most scalable) if we could find a way to do this tallying without storing the whole vocabulary in memory... Use an appropriate MapReduce design pattern to implement this efficiently! 


### Q8 Tasks:
* __a) make a plan:__  Fill in the docstrings for __`NaiveBayes/train_mapper.py`__ and __`NaiveBayes/train_reducer.py`__ to appropriately reflect the format that each script will input/output. [`HINT:` _the input files_ (`enronemail_1h.txt` & `chineseTrain.txt`) _have a prespecified format and your output file should match_ `NBmodel.txt` _so you really only have to decide on an internal format for Hadoop_].


* __b) implement it:__ Complete the code in __`NaiveBayes/train_mapper.py`__ and __`NaiveBayes/train_reducer.py`__ so that together they train a Multinomial Naive Bayes model __with no smoothing__. Make sure your end result is formatted correctly (see note above). Test your scripts independently and together (using `chineseTrain.txt` or test input of your own devising). When you are satisfied with your Python code design and run a Hadoop streaming command to run your job in parallel on the __chineseTrain.txt__. Confirm that your trained model matches your hand calculations from Question 6.


* __c) short response:__ We saw in Question 6 that adding Laplace smoothing (where the smoothing parameter $k=1$) makes our classifications less sensitve to rare words. However implementing this technique requires access to one additional piece of information that we had not previously used in our Naive Bayes training. What is that extra piece of information? [`HINT:` see equation 13.7 in Manning, Raghavan and Schutze].


* __d) short response:__ There are a couple of approaches that we could take to handle the extra piece of information you identified in `c`: 1) if we knew this extra information beforehand, we could provide it to our reducer as a configurable parameter for the vocab size dynamically (_where would we get it in the first place?_). Or 2) we could compute it in the reducer without storing any bulky information in memory but then we'd need some postprocessing or a second MapReduce job to complete the calculation (_why?_). Breifly explain what is non-ideal about each of these options. 


* __e) code + short response:__ Choose one of the 2 options above. State your choice & reasoning in the space below then use that strategy to complete the code in __`NaiveBayes/train_reducer_smooth.py`__. Test this alternate reducer then write and run a Hadoop streaming job to train an MNB model with smoothing on the Chinese example. Your results should match the model that we provided for you above (and the calculations in the textbook example). __IMPORTANT NOTE:__ For full credit on this question, your code must work with multiple reducers. 

    - [`HINT:` You will need to implement custom partitioning - [Total Order Sort Notebook](https://github.com/UCB-w261/main/tree/master/HelpfulResources/TotalSortGuide/_total-sort-guide-spark2.01-JAN27-2017.ipynb)] 

    - [`HINT:` Don't start from scratch with this one -- you can just copy over your reducer code from part `b` and make the needed modifications]. 



__IMPORTANT NOTE:__ For full credit on this question, your code must work with multiple reducers. [`HINT:`_You will need to implement custom partitioning - [Total Order Sort Notebook](https://github.com/UCB-w261/main/tree/master/HelpfulResources/TotalSortGuide/_total-sort-guide-spark2.01-JAN27-2017.ipynb)]


### Q8 Student Answers:
> __c)__ The extra information need when using Laplace smoothing is the variable  $B'$, which is the number of terms in the vocabulary.

> __d)__ The first option is not scalable because it does not lend itself to parallel computation.  Although it only requires one pass, it utilizes only one reducer, which is a bottleneck limiting this approach to small datasets.  

> The second option is not ideal in the sense that it requires two MapReduce jobs.  However, it is the method I recommend because it is scalable--it takes advantage of a parallel computation framework.

> __e)__ I am going with the second option.  Even though it's necessary to run two MapReduce jobs, it is far more scalable--the purpose of the Hadoop framework.


In [40]:
# part a - do your work in train_mapper.py and train_reducer.py then RUN THIS CELL AS IS
!chmod a+x NaiveBayes/train_mapper.py
!chmod a+x NaiveBayes/train_reducer.py
!echo "=========== MAPPER DOCSTRING ============"
!head -n 8 NaiveBayes/train_mapper.py | tail -n 6
!echo "=========== REDUCER DOCSTRING ============"
!head -n 8 NaiveBayes/train_reducer.py | tail -n 6

=========== MAPPER DOCSTRING ============
Mapper reads in text documents and emits word counts by class.
INPUT:                                                    
    DocID \t true_class \t subject \t body                
OUTPUT:                                                   
    partitionKey \t word \t class0_partialCount,class1_partialCount       
    
=========== REDUCER DOCSTRING ============
Reducer aggregates word counts by class and emits frequencies.

INPUT:
    partitionKey \t word \t class0_partialCount,class1_partialCount 
OUTPUT:
    ID \t word \t class0_count,class1count,P(word|class0),P(word|class1)


In [73]:
!cat NaiveBayes/train_mapper.py

#!/usr/bin/env python
"""
Mapper reads in text documents and emits word counts by class.
INPUT:                                                    
    DocID \t true_class \t subject \t body                
OUTPUT:                                                   
    partitionKey \t word \t class0_partialCount,class1_partialCount       
    

Instructions:
    You know what this script should do, go for it!
    (As a favor to the graders, please comment your code clearly!)
    
    A few reminders:
    1) To make sure your results match ours please be sure
       to use the same tokenizing that we have provided in
       all the other jobs:
         words = re.findall(r'[a-z]+', text-to-tokenize.lower())
         
    2) Don't forget to handle the various "totals" that you need
       for your conditional probabilities and class priors.
       
Partitioning:
    In order to send the totals to each reducer, we need to implement
    a custom partitioning strategy.
    
    We will gene

In [70]:
!cat NaiveBayes/train_reducer.py

#!/usr/bin/env python
"""
Reducer aggregates word counts by class and emits frequencies.

INPUT:
    partitionKey \t word \t class0_partialCount,class1_partialCount 
OUTPUT:
    ID \t word \t class0_count,class1count,P(word|class0),P(word|class1)
    
Instructions:
    Again, you are free to design a solution however you see 
    fit as long as your final model meets our required format
    for the inference job we designed in Question 8. Please
    comment your code clearly and concisely.
    
    A few reminders: 
    1) Don't forget to emit Class Priors (with the right key).
    2) In python2: 3/4 = 0 and 3/float(4) = 0.75
"""
##################### YOUR CODE HERE ####################


import re                                                   
import sys                                                  
import numpy as np      

from operator import itemgetter
import os

# initialize variables
cur_word = None
class0_count, class0_totalWords, class0_totalDocs = 0, 0, 0
class1_count

In [71]:
!cat NaiveBayes/train_reducer_smooth.py

#!/usr/bin/env python

import os
import sys                                                  
import numpy as np  

#################### YOUR CODE HERE ###################

# initialize variables
cur_word = None
class0_count, class0_totalDocs, class0_totalTerms, class0_totalWords = 0, 0, 0, 0
class1_count, class1_totalDocs, class1_totalTerms, class1_totalWords = 0, 0, 0, 0
totalTerms, totalWords, valueTemp0, valueTemp1 = 1, 0, None, None

# read input key-value pairs from standard input
for line in sys.stdin:
    
    line.rstrip('\n')
    pKey, word, value = line.split('\t',2)
    value_tuple = tuple(value.split (","))
    valueTemp0 = int(value_tuple[0])
    valueTemp1 = int(value_tuple[1])

    # tally counts from current key
    if word == cur_word:     
        class0_count += valueTemp0
        class1_count += valueTemp1
    else:
        # total words / class type word total
        if cur_word == "!totalWordCount":
            totalWords = class0_count + class1_count
          

__`part b starts here`:__ MNB _without_ Smoothing (training on Chinese Example Corpus).

In [38]:
# part b - write a unit test for your mapper here
!cat NaiveBayes/chineseTrain.txt | NaiveBayes/train_mapper.py | column -t

A  chinese              0,1
A  beijing              0,1
A  chinese              0,1
A  chinese              0,1
A  chinese              0,1
C  shanghai             0,1
A  chinese              0,1
B  macao                0,1
D  tokyo                1,0
B  japan                1,0
A  chinese              1,0
A  !totalWordCount      3,8
A  !totalDocClassCount  1,3
B  !totalWordCount      3,8
B  !totalDocClassCount  1,3
C  !totalWordCount      3,8
C  !totalDocClassCount  1,3
D  !totalWordCount      3,8
D  !totalDocClassCount  1,3


In [77]:
# part b - write a unit test for your reducer here
!cat NaiveBayes/chineseTestReducer.txt | NaiveBayes/train_reducer.py | column -t

alpha        1.0,  1.0,  0.3333333333333333,  0.125
beijing      1.0,  2.0,  0.3333333333333333,  0.25
beta         1.0,  0.0,  0.3333333333333333,  0.0
chinese      0.0,  5.0,  0.0,                 0.625
ClassPriors  1.0,  3.0,  0.25,                0.75


In [42]:
# part b - write a systems test for your mapper + reducer together here
!cat NaiveBayes/chineseTrain.txt | NaiveBayes/train_mapper.py | sort -k1,1 | NaiveBayes/train_reducer.py | column -t

beijing      0.0,  1.0,  0.0,                 0.125
chinese      1.0,  5.0,  0.3333333333333333,  0.625
japan        1.0,  0.0,  0.3333333333333333,  0.0
macao        0.0,  1.0,  0.0,                 0.125
shanghai     0.0,  1.0,  0.0,                 0.125
tokyo        1.0,  0.0,  0.3333333333333333,  0.0
ClassPriors  1.0,  3.0,  0.25,                0.75


In [43]:
# part b - clear (and name) an output directory in HDFS for your unsmoothed chinese NB model
!hdfs dfs -rm -r {HDFS_DIR}/nb-unsmoothed-chinese-output

Deleted /user/root/HW2/nb-unsmoothed-chinese-output


In [44]:
# part b - write your hadoop streaming job
!hdfs dfs -rm -r {HDFS_DIR}/nb-unsmoothed-chinese-output
!hadoop jar {JAR_FILE} \
  -D stream.num.map.output.key.fields=3 \
  -D stream.map.output.field.separator="\t" \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D mapreduce.partition.keycomparator.options="-k2,2" \
  -D mapreduce.partition.keypartitioner.options="-k1,1" \
  -files NaiveBayes/train_reducer.py,NaiveBayes/train_mapper.py \
  -mapper train_mapper.py \
  -reducer train_reducer.py \
  -input {HDFS_DIR}/chineseTrain.txt \
  -output {HDFS_DIR}/nb-unsmoothed-chinese-output \
  -cmdenv PATH={PATH} \
  -numReduceTasks 4 \
  -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner

rm: `/user/root/HW2//nb-unsmoothed-chinese-output': No such file or directory
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.16.2.jar] /tmp/streamjob7576241135749287986.jar tmpDir=null
21/09/18 20:07:36 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/09/18 20:07:36 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/09/18 20:07:37 INFO mapred.FileInputFormat: Total input paths to process : 1
21/09/18 20:07:37 INFO mapreduce.JobSubmitter: number of splits:2
21/09/18 20:07:37 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1631977959277_0002
21/09/18 20:07:37 INFO impl.YarnClientImpl: Submitted application application_1631977959277_0002
21/09/18 20:07:38 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1631977959277_0002/
21/09/18 20:07:38 INFO mapreduce.Job: Running job: job_1631977959277_0002
21/09/18 20:07:44 INFO mapreduce.Job: Job job_1631977959277_0002 running in u

In [45]:
# part b - extract your results (i.e. model) to a local file
!hdfs dfs -cat {HDFS_DIR}/nb-unsmoothed-chinese-output/part-000* > NaiveBayes/nbUnsmoothedChineseResults.txt

In [46]:
# part b - print your model so that we can confirm that it matches expected results
!cat NaiveBayes/nbUnsmoothedChineseResults.txt | column -t

tokyo        1.0,  0.0,  0.3333333333333333,  0.0
ClassPriors  1.0,  3.0,  0.25,                0.75
beijing      0.0,  1.0,  0.0,                 0.125
chinese      1.0,  5.0,  0.3333333333333333,  0.625
ClassPriors  1.0,  3.0,  0.25,                0.75
japan        1.0,  0.0,  0.3333333333333333,  0.0
macao        0.0,  1.0,  0.0,                 0.125
ClassPriors  1.0,  3.0,  0.25,                0.75
shanghai     0.0,  1.0,  0.0,                 0.125
ClassPriors  1.0,  3.0,  0.25,                0.75


In [74]:
!cat NaiveBayes/chineseTrain.txt

D1	1		Chinese Beijing Chinese
D2	1		Chinese Chinese Shanghai
D3	1		Chinese Macao
D4	0		Tokyo Japan Chinese


In [47]:
# view partitions
for p in range(4):
    print('='*10,f'PARTITION{p+1}','='*10)
    !hdfs dfs -cat {HDFS_DIR}/nb-unsmoothed-chinese-output/part-0000{p} | head -n 5

========== PARTITION1 ==========
tokyo	1.0, 0.0,      0.3333333333333333, 0.0
ClassPriors	1.0, 3.0,      0.25, 0.75
========== PARTITION2 ==========
beijing	0.0, 1.0,                  0.0, 0.125
chinese	1.0, 5.0,      0.3333333333333333, 0.625
ClassPriors	1.0, 3.0,      0.25, 0.75
========== PARTITION3 ==========
japan	1.0, 0.0,                  0.3333333333333333, 0.0
macao	0.0, 1.0,      0.0, 0.125
ClassPriors	1.0, 3.0,      0.25, 0.75
========== PARTITION4 ==========
shanghai	0.0, 1.0,      0.0, 0.125
ClassPriors	1.0, 3.0,      0.25, 0.75


__`part e starts here`:__ MNB _with_ Smoothing (training on Chinese Example Corpus).

In [78]:
# part e - write a unit test for your NEW reducer here
!chmod a+x NaiveBayes/train_reducer_smooth.py
!cat NaiveBayes/chineseTestReducer.txt | NaiveBayes/train_reducer_smooth.py | column -t

alpha        1.0,  1.0,  0.3333333333333333,  0.125
beijing      1.0,  2.0,  0.3333333333333333,  0.25
beta         1.0,  0.0,  0.3333333333333333,  0.0
chinese      0.0,  5.0,  0.0,                 0.625
ClassPriors  1.0,  3.0,  0.25,                0.75
!TotalTerms  4.0,  4.0,  3.0,                 8.0


In [58]:
# part e - write a systems test for your mapper + reducer together here
!cat NaiveBayes/chineseTrain.txt | NaiveBayes/train_mapper.py | sort -k1,1 | NaiveBayes/train_reducer_smooth.py | column -t

beijing      0.0,  1.0,  0.0,                 0.125
chinese      1.0,  5.0,  0.3333333333333333,  0.625
japan        1.0,  0.0,  0.3333333333333333,  0.0
macao        0.0,  1.0,  0.0,                 0.125
shanghai     0.0,  1.0,  0.0,                 0.125
tokyo        1.0,  0.0,  0.3333333333333333,  0.0
ClassPriors  1.0,  3.0,  0.25,                0.75
!TotalTerms  6.0,  6.0,  3.0,                 8.0


In [60]:
# part e - clear (and name) an output directory in HDFS for your SMOOTHED chinese NB 
!hdfs dfs -rm -r {HDFS_DIR}/nb-smoothed-chinese-output
# part e - write your hadoop streaming job
!hadoop jar {JAR_FILE} \
  -D stream.num.map.output.key.fields=3 \
  -D stream.map.output.field.separator="\t" \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D mapreduce.partition.keycomparator.options="-k2,2" \
  -D mapreduce.partition.keypartitioner.options="-k1,1" \
  -files NaiveBayes/train_reducer_smooth.py,NaiveBayes/train_mapper.py \
  -mapper train_mapper.py \
  -reducer train_reducer_smooth.py \
  -input {HDFS_DIR}/chineseTrain.txt \
  -output {HDFS_DIR}/nb-smoothed-chinese-output\
  -cmdenv PATH={PATH} \
  -numReduceTasks 4 \
  -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner

rm: `/user/root/HW2//nb-smoothed-chinese-output': No such file or directory
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.16.2.jar] /tmp/streamjob2236153097573439145.jar tmpDir=null
21/09/18 20:50:37 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/09/18 20:50:38 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/09/18 20:50:38 INFO mapred.FileInputFormat: Total input paths to process : 1
21/09/18 20:50:38 INFO mapreduce.JobSubmitter: number of splits:2
21/09/18 20:50:38 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1631977959277_0003
21/09/18 20:50:39 INFO impl.YarnClientImpl: Submitted application application_1631977959277_0003
21/09/18 20:50:39 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1631977959277_0003/
21/09/18 20:50:39 INFO mapreduce.Job: Running job: job_1631977959277_0003
21/09/18 20:50:46 INFO mapreduce.Job: Job job_1631977959277_0003 running in ube

In [62]:
# part e - extract your results (i.e. model) to a local file
!hdfs dfs -cat {HDFS_DIR}/nb-smoothed-chinese-output/part-000* > NaiveBayes/chineseResultsNBSmoothed.txt

In [80]:
# part e - print your model (above local file) so that we can confirm that it matches expected results
#!chmod a+x NaiveBayes/smoothNB_postProcess.py
!cat NaiveBayes/chineseResultsNBSmoothed.txt | sort -k1,1 | NaiveBayes/smoothNB_postProcess.py | column -t

ClassPriors  1.0,3.0,  0.25,0.75
beijing      0.0,      1.0,       0.1111111111111111,  0.14285714285714285
chinese      1.0,      5.0,       0.2222222222222222,  0.42857142857142855
japan        1.0,      0.0,       0.2222222222222222,  0.07142857142857142
macao        0.0,      1.0,       0.1111111111111111,  0.14285714285714285
shanghai     0.0,      1.0,       0.1111111111111111,  0.14285714285714285
tokyo        1.0,      0.0,       0.2222222222222222,  0.07142857142857142


# Question 9: Enron Ham/Spam NB Classifier & Results.

Fantastic work. We're finally ready to perform Spam Classification on the Enron Corpus. In this question you'll run the analysis you've developed, report its performance.

### Q9 Tasks:
* __a) train/test split:__ Run the provided code to split our Enron file into a training set and testing set then load them into HDFS. [`NOTE:` _Make sure you re calculate the vocab size for just the training set!_]

* __b) train 2 models:__ Write Hadoop Streaming jobs to train MNB Models on the training set with and without smoothing. Save your models to local files at __`NaiveBayes/Unsmoothed/NBmodel.txt`__ and __`NaiveBayes/Smoothed/NBmodel.txt`__. [`NOTE:` _This naming is important because we wrote our classification task so that it expects a file of that name... if this inelegance frustrates you there is an alternative that would involve a few adjustments to your code [read more about it here](http://www.tnoda.com/blog/2013-11-23)._] Finally run the checks that we provide to confirm that your results are correct.


* __c) code:__ Recall that we designed our classification job with just a mapper. An efficient way to report the performance of our models would be to simply add a reducer phase to this job and compute precision and recall right there. Complete the code in __`NaiveBayes/evaluation_reducer.py`__ and then write Hadoop jobs to evaluate your two models on the test set. Report their performance side by side. [`NOTE:` if you need a refresher on precision, recall and F1-score [Wikipedia](https://en.wikipedia.org/wiki/F1_score) is a good resource.]


* __d) short response:__ Compare the performance of your two models. What do you notice about the unsmoothed model's predictions? Can you guess why this is happening? Which evaluation measure do you think is most relevant in our use case? [`NOTE:` _Feel free to answer using your common sense but if you want more information on evaluating the classification task checkout_ [this blogpost](https://tryolabs.com/blog/2013/03/25/why-accuracy-alone-bad-measure-classification-tasks-and-what-we-can-do-about-it/
) or [this paper](http://www.flinders.edu.au/science_engineering/fms/School-CSEM/publications/tech_reps-research_artfcts/TRRA_2007.pdf
)]



### Q9 Student Answers:
> __d)__ The smoothed model has much higher accuracy compared to the unsmoothed model.  This is a result of the unsmoothed model handling terms not seen during training but seen during testing much better than the unsmoothed model.  Notice that the smoothed model has higher precision than the unsmoothed model, yet the smoothed model has a far superior recall.  In this case, the F1-score is most relevant.  These results indicate the smoothed model generalized much better than the unsmoothed model.



__Test/Train split__

In [68]:
# part a - test/train split (RUN THIS CELL AS IS)
!head -n 80 data/enronemail_1h.txt > data/enron_train.txt
!tail -n 20 data/enronemail_1h.txt > data/enron_test.txt
!hdfs dfs -copyFromLocal data/enron_train.txt {HDFS_DIR}
!hdfs dfs -copyFromLocal data/enron_test.txt {HDFS_DIR}

__Training__ (Enron MNB Model _without smoothing_ )

In [69]:
# part b -  Unsmoothed model (FILL IN THE MISSING CODE BELOW)

# clear the output directory
!hdfs dfs -rm -r {HDFS_DIR}/enron-model

# hadoop command
!hadoop jar {JAR_FILE} \
  -D stream.num.map.output.key.fields=3 \
  -D stream.map.output.field.separator="\t" \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D mapreduce.partition.keycomparator.options="-k2,2" \
  -D mapreduce.partition.keypartitioner.options="-k1,1" \
  -files NaiveBayes/train_reducer.py,NaiveBayes/train_mapper.py \
  -mapper train_mapper.py \
  -reducer train_reducer.py \
  -input {HDFS_DIR}/enron_train.txt \
  -output {HDFS_DIR}/enron-model \
  -cmdenv PATH={PATH} \
  -numReduceTasks 4 \
  -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner

# save the model locally
!mkdir NaiveBayes/Unsmoothed
!hdfs dfs -cat {HDFS_DIR}/enron-model/part-000* > NaiveBayes/Unsmoothed/NBmodel.txt

rm: `/user/root/HW2//enron-model': No such file or directory
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.16.2.jar] /tmp/streamjob5181463804026509919.jar tmpDir=null
21/09/19 00:58:04 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/09/19 00:58:04 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/09/19 00:58:05 INFO mapred.FileInputFormat: Total input paths to process : 1
21/09/19 00:58:05 INFO mapreduce.JobSubmitter: number of splits:2
21/09/19 00:58:05 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1631977959277_0004
21/09/19 00:58:05 INFO impl.YarnClientImpl: Submitted application application_1631977959277_0004
21/09/19 00:58:06 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1631977959277_0004/
21/09/19 00:58:06 INFO mapreduce.Job: Running job: job_1631977959277_0004
21/09/19 00:58:13 INFO mapreduce.Job: Job job_1631977959277_0004 running in uber mode : false


In [70]:
# part b - check your UNSMOOTHED model results (RUN THIS CELL AS IS)
!grep assistance NaiveBayes/Unsmoothed/NBmodel.txt
# EXPECTED OUTPUT: assistance	2,4,0.000172547666293,0.000296823983378

assistance	2.0, 4.0,                  0.0001725476662928134, 0.00029682398337785694


In [71]:
# part b - check your UNSMOOTHED model results (RUN THIS CELL AS IS)
!grep money NaiveBayes/Unsmoothed/NBmodel.txt
# EXPECTED OUTPUT: money	1,22,8.62738331464e-05,0.00163253190858

money	1.0, 22.0,                  8.62738331464067e-05, 0.001632531908578213


__Training__ (Enron MNB Model _with Laplace +1 smoothing_ )

In [81]:
# part b -  Smoothed model (FILL IN THE MISSING CODE BELOW)

# clear the output directory
!hdfs dfs -rm -r {HDFS_DIR}/smooth-model-preprocess

# hadoop command
!hadoop jar {JAR_FILE} \
  -D stream.num.map.output.key.fields=3 \
  -D stream.map.output.field.separator="\t" \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D mapreduce.partition.keycomparator.options="-k2,2" \
  -D mapreduce.partition.keypartitioner.options="-k1,1" \
  -files NaiveBayes/train_reducer_smooth.py,NaiveBayes/train_mapper.py \
  -mapper train_mapper.py \
  -reducer train_reducer_smooth.py \
  -input {HDFS_DIR}/enron_train.txt \
  -output {HDFS_DIR}/smooth-model-preprocess \
  -cmdenv PATH={PATH} \
  -numReduceTasks 4 \
  -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner

# save the model locally
!mkdir NaiveBayes/Unsmoothed
!hdfs dfs -cat {HDFS_DIR}/smooth-model/part-000* > NaiveBayes/Smoothed/NBmodel-preprocess.txt

Deleted /user/root/HW2/smooth-model-preprocess
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.16.2.jar] /tmp/streamjob1120084469177153423.jar tmpDir=null
21/09/19 01:08:03 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/09/19 01:08:04 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/09/19 01:08:04 INFO mapred.FileInputFormat: Total input paths to process : 1
21/09/19 01:08:04 INFO mapreduce.JobSubmitter: number of splits:2
21/09/19 01:08:04 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1631977959277_0013
21/09/19 01:08:05 INFO impl.YarnClientImpl: Submitted application application_1631977959277_0013
21/09/19 01:08:05 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1631977959277_0013/
21/09/19 01:08:05 INFO mapreduce.Job: Running job: job_1631977959277_0013
21/09/19 01:08:11 INFO mapreduce.Job: Job job_1631977959277_0013 running in uber mode : false
21/09/19 01:08

In [79]:
# part b -  Smoothed model postprocessing

# clear the output directory
!hdfs dfs -rm -r {HDFS_DIR}/smooth-model

# hadoop command
!hadoop jar {JAR_FILE} \
  -files NaiveBayes/smoothNB_postProcess.py\
  -mapper /bin/cat \
  -reducer smoothNB_postProcess.py \
  -input {HDFS_DIR}/smooth-model-preprocess \
  -output {HDFS_DIR}/smooth-model \
  -cmdenv PATH={PATH} \
  -numReduceTasks 1 \

# save the model locally
# !mkdir NaiveBayes/Smoothed
!hdfs dfs -cat {HDFS_DIR}/smooth-model/part-000* > NaiveBayes/Smoothed/NBmodel.txt

Deleted /user/root/HW2/smooth-model
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.16.2.jar] /tmp/streamjob999650093935238747.jar tmpDir=null
21/09/22 05:56:05 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/09/22 05:56:05 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/09/22 05:56:06 INFO mapred.FileInputFormat: Total input paths to process : 4
21/09/22 05:56:06 INFO mapreduce.JobSubmitter: number of splits:4
21/09/22 05:56:06 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1632276588378_0011
21/09/22 05:56:06 INFO impl.YarnClientImpl: Submitted application application_1632276588378_0011
21/09/22 05:56:06 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1632276588378_0011/
21/09/22 05:56:06 INFO mapreduce.Job: Running job: job_1632276588378_0011
21/09/22 05:56:13 INFO mapreduce.Job: Job job_1632276588378_0011 running in uber mode : false
21/09/22 05:56:13 INFO map

In [84]:
# part b - check your SMOOTHED model results (RUN THIS CELL AS IS)
!grep assistance NaiveBayes/Smoothed/NBmodel.txt
# EXPECTED OUTPUT: assistance	2,4,0.000185804533631,0.000277300205202

assistance	2.0, 4.0,                  0.0001858045336306206, 0.00027730020520215184


In [85]:
# part b - check your SMOOTHED model results (RUN THIS CELL AS IS)
!grep money NaiveBayes/Smoothed/NBmodel.txt
# EXPECTED OUTPUT: money	1,22,0.000123869689087,0.00127558094393

money	1.0, 22.0,                  0.0001238696890870804, 0.0012755809439298986


__Evaluation__

In [86]:
# part c - write your code in NaiveBayes/evaluation_reducer.py then RUN THIS
!chmod a+x NaiveBayes/evaluation_reducer.py

In [87]:
!cat NaiveBayes/evaluation_reducer.py

#!/usr/bin/env python
"""
Reducer to calculate precision and recall as part
of the inference phase of Naive Bayes.
INPUT:
    ID \t true_class \t P(ham|doc) \t P(spam|doc) \t predicted_class
OUTPUT:
    precision \t ##
    recall \t ##
    accuracy \t ##
    F-score \t ##
         
Instructions:
    Complete the missing code to compute these^ four
    evaluation measures for our classification task.
    
    Note: if you have no True Positives you will not 
    be able to compute the F1 score (and maybe not 
    precision/recall). Your code should handle this 
    case appropriately feel free to interpret the 
    "output format" above as a rough suggestion. It
    may be helpful to also print the counts for true
    positives, false positives, etc.
"""
import sys

# initialize counters
FP = 0.0 # false positives
FN = 0.0 # false negatives
TP = 0.0 # true positives
TN = 0.0 # true negatives

# read from STDIN
for line in sys.stdin:
    # parse input
    docID, class_, pHam, pSpam, pred

In [89]:
# part c - unit test your evaluation job on the chinese model (RUN THIS CELL AS IS)
!cat NaiveBayes/chineseTest.txt | NaiveBayes/classify_mapper.py 
print("")
!cat NaiveBayes/chineseTest.txt | NaiveBayes/classify_mapper.py | NaiveBayes/evaluation_reducer.py

d5	1	-8.90668134500626	-8.10769031284611	1
d6	1	-5.780743515794329	-4.179502370564408	1
d7	0	-6.591673732011658	-7.511706880737811	0
d8	0	-4.394449154674438	-5.565796731681498	0

d5	1	-8.90668134500626	-8.10769031284611	 True
d6	1	-5.780743515794329	-4.179502370564408	 True
d7	0	-6.591673732011658	-7.511706880737811	 True
d8	0	-4.394449154674438	-5.565796731681498	 True
True Positives	2.0
True Negatives	2.0
False Positives	0.0
False Negatives	0.0
Precision	1.0
Recall	1.0
Accuracy	1.0
F-score	1.0


In [94]:
# part c - Evaluate the UNSMOOTHED Model Here (FILL IN THE MISSING CODE)

# clear output directory
!hdfs dfs -rm -r {HDFS_DIR}/unsmoothed-evaluation
# hadoop job
!hadoop jar {JAR_FILE} \
  -files NaiveBayes/evaluation_reducer.py,NaiveBayes/classify_mapper.py,NaiveBayes/Unsmoothed/NBmodel.txt \
  -mapper classify_mapper.py \
  -reducer evaluation_reducer.py \
  -input {HDFS_DIR}/enron_test.txt \
  -output {HDFS_DIR}/unsmoothed-evaluation \
  -cmdenv PATH={PATH} \
  -numReduceTasks 1 \

# retrieve results locally
!hdfs dfs -cat {HDFS_DIR}/unsmoothed-evaluation/part-000* > NaiveBayes/Unsmoothed/results.txt

rm: `/user/root/HW2/unsmoothed-evaluation': No such file or directory
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.16.2.jar] /tmp/streamjob7819299292078338768.jar tmpDir=null
21/09/19 01:34:52 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/09/19 01:34:52 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/09/19 01:34:53 INFO mapred.FileInputFormat: Total input paths to process : 1
21/09/19 01:34:53 INFO mapreduce.JobSubmitter: number of splits:2
21/09/19 01:34:53 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1631977959277_0018
21/09/19 01:34:53 INFO impl.YarnClientImpl: Submitted application application_1631977959277_0018
21/09/19 01:34:53 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1631977959277_0018/
21/09/19 01:34:53 INFO mapreduce.Job: Running job: job_1631977959277_0018
21/09/19 01:35:00 INFO mapreduce.Job: Job job_1631977959277_0018 running in uber mode

In [95]:
# part c - Evaluate the SMOOTHED Model Here (FILL IN THE MISSING CODE)

# clear output directory
!hdfs dfs -rm -r {HDFS_DIR}/smoothed-evaluation
# hadoop job
!hadoop jar {JAR_FILE} \
  -files NaiveBayes/evaluation_reducer.py,NaiveBayes/classify_mapper.py,NaiveBayes/Smoothed/NBmodel.txt \
  -mapper classify_mapper.py \
  -reducer evaluation_reducer.py \
  -input {HDFS_DIR}/enron_test.txt \
  -output {HDFS_DIR}/smoothed-evaluation \
  -cmdenv PATH={PATH} \
  -numReduceTasks 1 \

# retrieve results locally
!hdfs dfs -cat {HDFS_DIR}/smoothed-evaluation/part-000* > NaiveBayes/Smoothed/results.txt

Deleted /user/root/HW2/smoothed-evaluation
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.16.2.jar] /tmp/streamjob429253816486911156.jar tmpDir=null
21/09/19 01:35:24 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/09/19 01:35:24 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/09/19 01:35:24 INFO mapred.FileInputFormat: Total input paths to process : 1
21/09/19 01:35:24 INFO mapreduce.JobSubmitter: number of splits:2
21/09/19 01:35:25 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1631977959277_0019
21/09/19 01:35:25 INFO impl.YarnClientImpl: Submitted application application_1631977959277_0019
21/09/19 01:35:25 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1631977959277_0019/
21/09/19 01:35:25 INFO mapreduce.Job: Running job: job_1631977959277_0019
21/09/19 01:35:31 INFO mapreduce.Job: Job job_1631977959277_0019 running in uber mode : false
21/09/19 01:35:31 I

In [96]:
# part c - display results 
# NOTE: feel free to modify the tail commands to match the format of your results file
print('=========== UNSMOOTHED MODEL ============')
!tail -n 9 NaiveBayes/Unsmoothed/results.txt
print('=========== SMOOTHED MODEL ============')
!tail -n 9 NaiveBayes/Smoothed/results.txt

=========== UNSMOOTHED MODEL ============
0018.2003-12-18.gp	1	-inf	-inf	 False
True Positives	1.0
True Negatives	9.0
False Positives	0.0
False Negatives	10.0
Precision	1.0
Recall	0.09090909090909091
Accuracy	0.5
F-score	0.16666666666666669
=========== SMOOTHED MODEL ============
0018.2003-12-18.gp	1	-2754.1808545502745	-2687.628142401689	 True
True Positives	11.0
True Negatives	6.0
False Positives	3.0
False Negatives	0.0
Precision	0.7857142857142857
Recall	1.0
Accuracy	0.85
F-score	0.88


__`EXPECTED RESULTS:`__ 
<table>
<th>Unsmoothed Model</th>
<th>Smoothed Model</th>
<tr>
<td><pre>
# Documents:	20
True Positives:	1
True Negatives:	9
False Positives:	0
False Negatives:	10
Accuracy	0.5
Precision	1.0
Recall	0.0909
F-Score	0.1666
</pre></td>
<td><pre>
# Documents:	20
True Positives:	11
True Negatives:	6
False Positives:	3
False Negatives:	0
Accuracy	0.85
Precision	0.7857
Recall	1.0
F-Score	0.88
</pre></td>
</tr>
</table>

__`NOTE:`__ _Don't be too disappointed if these seem low to you. We've trained and tested on a very very small corpus... bigger datasets coming soon!_

# Question 10: Custom Partitioning and Secondary Sort

Now that we have our model, we can analyse the results and think about future improvements.

### Q10 Tasks:

* __a) code + short response:__ Let's look at the top ten words with the highest conditional probability in `Spam` and in `Ham`. We'll do this by writing a Hadoop job that sorts the model file (`NaiveBayes/Smoothed/NBmodel.py`). Normally we'd have to run two jobs -- one that sorts on $P(word|ham)$ and another that sorts on $P(word|spam)$. However if we slighly modify the data format in the model file then we can get the top words in each class with just one job. We've written a mapper that will do just this for you. Read through __`NaiveBayes/model_sort_mapper.py`__ and then briefly explain how this mapper will allow us to partition and sort our model file. Write a Hadoop job that uses our mapper and `/bin/cat` for a reducer to partition and sort. Print out the top 10 words in each class (where 'top' == highest conditional probability).[`HINT:` _this should remind you a lot of what we did in Question 6._]


* __b) short response:__ What do you notice about the 'top words' we printed in `a`? How would increasing the smoothing parameter 'k' affect the probabilities for the top words that you identified for 'a'. How would they affect the probabilities of words that occur much more in one class than another? In summary, how does the smoothing parameter 'k' affect the bias and the variance of our model. [`NOTE:` _you do not need to code anything for this task, but if you are struggling with it you could try changing 'k' and see what happens to the test set. We don't recommend doing this exploration with the Enron data because it will be harder to see the impact with such a big vocabulary_]

### Q10 Student Answers:
> __a)__ The mapper receives as input the key-value pair of:  
key - word  
value - count ham, count spam, conditional probability ham, conditional probability spam  
It then pulls out the class and conditional probability for ham and spam separately.  Finally, it emits the record with the class and the corresponding conditional probability.   

> __b)__ Higher values of the smoothing parameter 'k' will reduce the variance of the model but increase its bias.

In [126]:
!chmod a+x NaiveBayes/model_sort_mapper.py
!cat NBmodel.txt | NaiveBayes/model_sort_mapper.py | column -t > NaiveBayes/NBmodel_sort_mapper.txt

In [128]:
!cat NaiveBayes/NBmodel_sort_mapper.txt

beijing   0.0,1.0,0.111111111111,0.142857142857   ham   0.111111111111
beijing   0.0,1.0,0.111111111111,0.142857142857   spam  0.142857142857
chinese   1.0,5.0,0.222222222222,0.428571428571   ham   0.222222222222
chinese   1.0,5.0,0.222222222222,0.428571428571   spam  0.428571428571
tokyo     1.0,0.0,0.222222222222,0.0714285714286  ham   0.222222222222
tokyo     1.0,0.0,0.222222222222,0.0714285714286  spam  0.0714285714286
shanghai  0.0,1.0,0.111111111111,0.142857142857   ham   0.111111111111
shanghai  0.0,1.0,0.111111111111,0.142857142857   spam  0.142857142857
japan     1.0,0.0,0.222222222222,0.0714285714286  ham   0.222222222222
japan     1.0,0.0,0.222222222222,0.0714285714286  spam  0.0714285714286
macao     0.0,1.0,0.111111111111,0.142857142857   ham   0.111111111111
macao     0.0,1.0,0.111111111111,0.142857142857   spam  0.142857142857


In [187]:
!sort -k4,4 NaiveBayes/NBmodel_sort_mapper.txt 

beijing   0.0,1.0,0.111111111111,0.142857142857   ham   0.111111111111
macao     0.0,1.0,0.111111111111,0.142857142857   ham   0.111111111111
shanghai  0.0,1.0,0.111111111111,0.142857142857   ham   0.111111111111
chinese   1.0,5.0,0.222222222222,0.428571428571   ham   0.222222222222
japan     1.0,0.0,0.222222222222,0.0714285714286  ham   0.222222222222
tokyo     1.0,0.0,0.222222222222,0.0714285714286  ham   0.222222222222
japan     1.0,0.0,0.222222222222,0.0714285714286  spam  0.0714285714286
tokyo     1.0,0.0,0.222222222222,0.0714285714286  spam  0.0714285714286
beijing   0.0,1.0,0.111111111111,0.142857142857   spam  0.142857142857
macao     0.0,1.0,0.111111111111,0.142857142857   spam  0.142857142857
shanghai  0.0,1.0,0.111111111111,0.142857142857   spam  0.142857142857
chinese   1.0,5.0,0.222222222222,0.428571428571   spam  0.428571428571


In [124]:
!sort -t, -k3n,3 -k1,1 NBmodel.txt | column -t

beijing      0.0,1.0,0.111111111111,0.142857142857
macao        0.0,1.0,0.111111111111,0.142857142857
shanghai     0.0,1.0,0.111111111111,0.142857142857
chinese      1.0,5.0,0.222222222222,0.428571428571
japan        1.0,0.0,0.222222222222,0.0714285714286
tokyo        1.0,0.0,0.222222222222,0.0714285714286
ClassPriors  1.0,3.0,0.25,0.75


In [171]:
!hdfs dfs -cat {HDFS_DIR}/smooth-model/part-00000 | head | column -t

ClassPriors  47.0,33.0,  0.5875,0.4125
a            168.0,      274.0,         0.010466988727858293,    0.015251511286118352
ab           4.0,        0.0,           0.00030967422271770096,  5.546004104043037e-05
able         3.0,        8.0,           0.0002477393781741608,   0.0004991403693638733
abn          0.0,        1.0,           6.19348445435402e-05,    0.00011092008208086075
about        8.0,        36.0,          0.0005574136008918618,   0.0020520215184959235
above        2.0,        5.0,           0.0001858045336306206,   0.00033276024624258224
absent       1.0,        0.0,           0.0001238696890870804,   5.546004104043037e-05
absenteeism  0.0,        1.0,           6.19348445435402e-05,    0.00011092008208086075
absolute     0.0,        1.0,           6.19348445435402e-05,    0.00011092008208086075
cat: Unable to write to output stream.


In [181]:
!hdfs dfs -copyFromLocal {HOME_DIR}/NBmodel.txt {HDFS_DIR}/NBmodel.txt

In [10]:
# part a - write your Hadoop job here (sort smoothed model on P(word|class))

# clear output directory
!hdfs dfs -rm -r {HDFS_DIR}/smoothed-sort
# hadoop 
!hadoop jar {JAR_FILE} \
  -D stream.num.map.output.key.fields=4 \
  -D stream.map.output.field.separator="\t" \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D mapreduce.partition.keycomparator.options="-k4,4nr" \
  -D mapreduce.partition.keypartitioner.options="-k3,3" \
  -files NaiveBayes/model_sort_mapper.py \
  -mapper model_sort_mapper.py \
  -reducer /bin/cat \
  -input {HDFS_DIR}/NBmodel.txt \
  -output {HDFS_DIR}/smoothed-sort \
  -cmdenv PATH={PATH} \
  -numReduceTasks 2 \
  -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner



rm: `/user/root/HW2/smoothed-sort': No such file or directory
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.16.2.jar] /tmp/streamjob6217861309351116323.jar tmpDir=null
21/09/22 06:58:55 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/09/22 06:58:55 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/09/22 06:58:56 INFO mapred.FileInputFormat: Total input paths to process : 1
21/09/22 06:58:56 INFO mapreduce.JobSubmitter: number of splits:2
21/09/22 06:58:56 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1632276588378_0034
21/09/22 06:58:56 INFO impl.YarnClientImpl: Submitted application application_1632276588378_0034
21/09/22 06:58:56 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1632276588378_0034/
21/09/22 06:58:56 INFO mapreduce.Job: Running job: job_1632276588378_0034
21/09/22 06:59:04 INFO mapreduce.Job: Job job_1632276588378_0034 running in uber mode : false

In [11]:
# retrieve results locally
!hdfs dfs -cat {HDFS_DIR}/smoothed-sort/part-000* > NaiveBayes/Smoothed/smoothed_sort.txt

In [12]:
# part b - print top words in each class
for p in range(2):
    print('='*10,f'PARTITION{p+1}','='*10)
    !hdfs dfs -cat {HDFS_DIR}/smoothed-sort/part-0000{p} | head -n 10

========== PARTITION1 ==========
japan	1.0,0.0,0.222222222222,0.0714285714286	ham	0.222222222222	
tokyo	1.0,0.0,0.222222222222,0.0714285714286	ham	0.222222222222	
chinese	1.0,5.0,0.222222222222,0.428571428571	ham	0.222222222222	
shanghai	0.0,1.0,0.111111111111,0.142857142857	ham	0.111111111111	
beijing	0.0,1.0,0.111111111111,0.142857142857	ham	0.111111111111	
macao	0.0,1.0,0.111111111111,0.142857142857	ham	0.111111111111	
========== PARTITION2 ==========
chinese	1.0,5.0,0.222222222222,0.428571428571	spam	0.428571428571	
shanghai	0.0,1.0,0.111111111111,0.142857142857	spam	0.142857142857	
beijing	0.0,1.0,0.111111111111,0.142857142857	spam	0.142857142857	
macao	0.0,1.0,0.111111111111,0.142857142857	spam	0.142857142857	
japan	1.0,0.0,0.222222222222,0.0714285714286	spam	0.0714285714286	
tokyo	1.0,0.0,0.222222222222,0.0714285714286	spam	0.0714285714286	


### Congratulations, you have completed HW2! Please refer to the readme for submission instructions.

If you would like to provide feedback regarding this homework, please use the survey at: https://docs.google.com/forms/d/e/1FAIpQLSce9feiQeSkdP43A0ZYui1tMGIBfLfzb0rmgToQeZD9bXXX8Q/viewform

In [6]:
!cat NBmodel.txt | head -10

beijing	0.0,1.0,0.111111111111,0.142857142857
chinese	1.0,5.0,0.222222222222,0.428571428571
tokyo	1.0,0.0,0.222222222222,0.0714285714286
shanghai	0.0,1.0,0.111111111111,0.142857142857
ClassPriors	1.0,3.0,0.25,0.75
japan	1.0,0.0,0.222222222222,0.0714285714286
macao	0.0,1.0,0.111111111111,0.142857142857
